In [2]:
import pandas as pd
import sqlite3

/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_42855/2059591896.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [56]:
connection = sqlite3.connect("olist.db")

df_reviews = pd.read_sql_query("SELECT * FROM Reviews",connection)

connection.close()

In [12]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98410 entries, 0 to 98409
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                98410 non-null  object
 1   order_id                 98410 non-null  object
 2   review_score             98410 non-null  int64 
 3   review_comment_title     98410 non-null  object
 4   review_comment_message   98410 non-null  object
 5   review_creation_date     98410 non-null  object
 6   review_answer_timestamp  98410 non-null  object
 7   timestamp_field_7        26 non-null     object
dtypes: int64(1), object(7)
memory usage: 6.0+ MB


In [15]:
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,,,2018-04-13 00:00:00,2018-04-16T00:39:37Z
1,9a0abbb668bafb95a6d2b05db43284c4,d7bd0e4afdf94846eb73642b4e3e75c3,3,,,2017-04-30 00:00:00,2017-05-03T00:02:22Z
2,fdbdb2629a7cde0f66657acc92084e7f,70a752414a13d09cc1f2b437b914b28e,3,,,2017-09-29 00:00:00,2017-10-02T01:12:49Z
3,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,,,2018-01-18 00:00:00,2018-01-18T21:46:59Z
4,23f75a37effc35d9a915b4e1ad483793,2eaf8e099d871cd5c22b83b5ea8f6e0e,4,,,2018-03-28 00:00:00,2018-03-30T15:10:55Z


# 1. Nettoyage de la table Reviews

## 1.1 Gestion des valeurs manquantes

In [57]:
df_reviews = df_reviews.drop(["timestamp_field_7"],axis=1)

In [14]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98410 entries, 0 to 98409
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                98410 non-null  object
 1   order_id                 98410 non-null  object
 2   review_score             98410 non-null  int64 
 3   review_comment_title     98410 non-null  object
 4   review_comment_message   98410 non-null  object
 5   review_creation_date     98410 non-null  object
 6   review_answer_timestamp  98410 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


## 1.2 Gestion des doublons

In [58]:
df_reviews = df_reviews.drop_duplicates(['order_id','review_score','review_comment_title','review_comment_message','review_creation_date'])

In [33]:
df_reviews.shape

(98371, 7)

### analyse

In [19]:
df_reviews.drop_duplicates().shape

(98410, 7)

In [20]:
df_reviews.drop_duplicates(['order_id']).shape

(98167, 7)

In [21]:
df_reviews.drop_duplicates(['review_id']).shape

(98410, 7)

In [24]:
df_reviews[["order_id"]].value_counts().sort_values(ascending=False)

order_id                        
7b1219cc26d69c357880055b52d87960    2
b1e45a9d2437f4116c3ff5de92d2a527    2
76ac5a9b38c945eb910681d171e98c55    2
3ab974ac744bd19a111d5e19f0a90649    2
aa20064eef8808d62ed9ac5286ec0796    2
                                   ..
ff1ea7be87e1ca3d2ae5b77eaa6dab69    1
ff1f5f13541ce8f1a0755596df027a90    1
ff1f8f328e5a59ac754240b6dd31bcb4    1
ff207aa3493754ffcea819b8c0c0dbb5    1
fffe41c64501cc87c801fd61db3f6244    1
Name: count, Length: 98167, dtype: int64

In [31]:
df_reviews.loc[df_reviews["order_id"]=="aa20064eef8808d62ed9ac5286ec0796",:]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
6796,662d2c533d56243557c12699eff9d5a3,aa20064eef8808d62ed9ac5286ec0796,5,,,2017-04-14 00:00:00,2017-04-16 19:41:32
73794,d83391dee674f4df74b3e72e31774030,aa20064eef8808d62ed9ac5286ec0796,5,,,2017-04-20 00:00:00,2017-04-23 11:14:56


## 1.3 Gestion des types de données

### 1.3.1 COnversion des typestamp

In [37]:
df_reviews.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [63]:
df_reviews.review_creation_date = pd.to_datetime(df_reviews['review_creation_date'], format= '%Y-%m-%d %H:%M:%S', errors="coerce")

In [64]:
df_reviews.review_answer_timestamp = pd.to_datetime(df_reviews['review_answer_timestamp'], format= '%Y-%m-%d %H:%M:%S', errors="coerce")

In [86]:
df_reviews.dtypes['review_creation_date']

dtype('<M8[ns]')

In [87]:
from numpy import dtype
df_reviews.dtypes['review_creation_date'] == dtype('<M8[ns]')


False

In [65]:
df_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
98405,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,,,2018-07-07,2018-07-14 17:18:30
98406,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,,,2017-12-09,2017-12-11 20:06:42
98407,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43
98408,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,,,2018-07-01,2018-07-02 12:59:13
98409,efe49f1d6f951dd88b51e6ccd4cc548f,90531360ecb1eec2a1fbb265a0db0508,1,,"meu produto chegou e ja tenho que devolver, po...",2017-07-03,2017-07-03 21:01:49


In [66]:
df_reviews.shape

(98371, 7)

In [67]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98371 entries, 0 to 98409
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                98371 non-null  object        
 1   order_id                 98371 non-null  object        
 2   review_score             98371 non-null  int64         
 3   review_comment_title     98371 non-null  object        
 4   review_comment_message   98371 non-null  object        
 5   review_creation_date     98368 non-null  datetime64[ns]
 6   review_answer_timestamp  98344 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 6.0+ MB


### 1.3.2 Remplacement des dates manquantes

In [70]:
df_reviews = df_reviews.dropna(subset=['review_creation_date','review_answer_timestamp'])

In [71]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98344 entries, 27 to 98409
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                98344 non-null  object        
 1   order_id                 98344 non-null  object        
 2   review_score             98344 non-null  int64         
 3   review_comment_title     98344 non-null  object        
 4   review_comment_message   98344 non-null  object        
 5   review_creation_date     98344 non-null  datetime64[ns]
 6   review_answer_timestamp  98344 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 6.0+ MB


In [69]:
df_reviews.loc[df_reviews['review_answer_timestamp'].isna(),:]['review_score'].value_counts()

review_score
5    16
4     6
1     2
3     2
2     1
Name: count, dtype: int64

## 1.4 Consistance des données

In [73]:
df_reviews['review_score'].value_counts()

review_score
5    56866
4    18993
1    11277
3     8095
2     3113
Name: count, dtype: int64

# 2 Jointure et nettoyage de la table Orders

In [4]:
connection = sqlite3.connect("olist.db")

df = pd.read_sql_query("SELECT * FROM CleanDataset",connection)



connection.close()

In [3]:
connection = sqlite3.connect("olist.db")

df_reviews = pd.read_sql_query("SELECT * FROM Clean",connection)

df_orders = pd.read_sql_query("SELECT * FROM Orders",connection)

connection.close()


DatabaseError: Execution failed on sql 'SELECT * FROM ReviewsClean': no such table: ReviewsClean

In [4]:
df = df_reviews.merge(df_orders, how='left', on ='order_id')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98344 entries, 0 to 98343
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   review_id                      98344 non-null  object
 1   order_id                       98344 non-null  object
 2   review_score                   98344 non-null  int64 
 3   review_comment_title           98344 non-null  object
 4   review_comment_message         98344 non-null  object
 5   review_creation_date           98344 non-null  object
 6   review_answer_timestamp        98344 non-null  object
 7   customer_id                    98344 non-null  object
 8   order_status                   98344 non-null  object
 9   order_purchase_timestamp       98344 non-null  object
 10  order_approved_at              98344 non-null  object
 11  order_delivered_carrier_date   98344 non-null  object
 12  order_delivered_customer_date  95542 non-null  object
 13  o

In [6]:
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,fe3db7c069d694bab50cc43463f91608,2ca73e2ff9e3a186ad1e1ffb9b1d9c10,5,,A compra foi realizada facilmente.\r\nA entreg...,2018-03-23 00:00:00,2018-04-01 00:27:51,e76f777f29e83b393d786accf49e7955,delivered,2018-03-18 21:01:32,2018-03-18 21:15:36,2018-03-19 19:52:47,2018-03-23 00:32:34,2018-04-11 00:00:00
1,e07eee7ad7666276bd8b27fbd83fd1ff,68b3cbc68e4cd1fe19bec35f1d3bb7ee,5,,relógio muito bonito e barato.,2018-03-29 00:00:00,2018-04-02 14:19:31,1a814d4c8923ef3a0e978777f00b717c,delivered,2018-03-10 00:42:20,2018-03-10 01:21:04,2018-03-12 21:05:34,2018-03-28 18:13:06,2018-03-29 00:00:00
2,2c5e27fc178bde7ac173c9c62c31b070,0ce9a24111d850192a933fcaab6fbad3,1,,Não gostei ! Comprei gato por lebre,2017-12-13 00:00:00,2017-12-16 07:14:07,5bb8de60ca2ca8b01a5ce471802fe10b,delivered,2017-11-24 01:40:48,2017-11-24 01:49:34,2017-12-06 15:19:09,2017-12-13 00:28:44,2017-12-19 00:00:00
3,540e7bbb2d06cfb7f85f3a88ba7ac97f,cf73e2cb1f4a9480ed70c154da3d954a,5,,,2018-01-18 00:00:00,2018-01-18 19:12:30,ef1ed1347404166e73e8f3977ffcbca0,delivered,2018-01-11 11:23:09,2018-01-12 02:38:33,2018-01-13 01:07:00,2018-01-17 14:36:38,2018-02-01 00:00:00
4,6a18a4001ae7fd911b3b9fb13686d105,8ba3f3a9292fbc22a8b7dff752bbcd37,5,,,2017-07-19 00:00:00,2017-07-20 10:38:48,0315a87fba4dab4865f2071777b98e87,delivered,2017-07-07 10:07:16,2017-07-07 10:23:34,2017-07-08 11:46:44,2017-07-18 23:16:44,2017-07-31 00:00:00


In [7]:
df.order_id.value_counts()

order_id
0d6cc2219ef268358eac3c49b47abca9    2
29f5dd847b7db645b7ce75cc49ddb612    2
25cb912c6742e486c6988aaf2c8f221b    2
bf593abe2928b23b60cee1ff3471a4dd    2
b1e45a9d2437f4116c3ff5de92d2a527    2
                                   ..
c25915645324f8ebbfa289b765eb6e8b    1
19a870c81a3ab57258a090b0976ec5ee    1
369d7f9f1c7cf25b5b840af3ddcc105b    1
58be617c22afd358984d890ce826e0c9    1
90531360ecb1eec2a1fbb265a0db0508    1
Name: count, Length: 98140, dtype: int64

## 2.1 Gestion des valeurs manquantes

Pas de NA a priori

## 2.2 Gestion des doublons

Pas de doublon de ReviewsCLean donc comme on fait un left, on aura pas de doublon dans df

## 2.3 Gestion des types

In [8]:
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,fe3db7c069d694bab50cc43463f91608,2ca73e2ff9e3a186ad1e1ffb9b1d9c10,5,,A compra foi realizada facilmente.\r\nA entreg...,2018-03-23 00:00:00,2018-04-01 00:27:51,e76f777f29e83b393d786accf49e7955,delivered,2018-03-18 21:01:32,2018-03-18 21:15:36,2018-03-19 19:52:47,2018-03-23 00:32:34,2018-04-11 00:00:00
1,e07eee7ad7666276bd8b27fbd83fd1ff,68b3cbc68e4cd1fe19bec35f1d3bb7ee,5,,relógio muito bonito e barato.,2018-03-29 00:00:00,2018-04-02 14:19:31,1a814d4c8923ef3a0e978777f00b717c,delivered,2018-03-10 00:42:20,2018-03-10 01:21:04,2018-03-12 21:05:34,2018-03-28 18:13:06,2018-03-29 00:00:00
2,2c5e27fc178bde7ac173c9c62c31b070,0ce9a24111d850192a933fcaab6fbad3,1,,Não gostei ! Comprei gato por lebre,2017-12-13 00:00:00,2017-12-16 07:14:07,5bb8de60ca2ca8b01a5ce471802fe10b,delivered,2017-11-24 01:40:48,2017-11-24 01:49:34,2017-12-06 15:19:09,2017-12-13 00:28:44,2017-12-19 00:00:00
3,540e7bbb2d06cfb7f85f3a88ba7ac97f,cf73e2cb1f4a9480ed70c154da3d954a,5,,,2018-01-18 00:00:00,2018-01-18 19:12:30,ef1ed1347404166e73e8f3977ffcbca0,delivered,2018-01-11 11:23:09,2018-01-12 02:38:33,2018-01-13 01:07:00,2018-01-17 14:36:38,2018-02-01 00:00:00
4,6a18a4001ae7fd911b3b9fb13686d105,8ba3f3a9292fbc22a8b7dff752bbcd37,5,,,2017-07-19 00:00:00,2017-07-20 10:38:48,0315a87fba4dab4865f2071777b98e87,delivered,2017-07-07 10:07:16,2017-07-07 10:23:34,2017-07-08 11:46:44,2017-07-18 23:16:44,2017-07-31 00:00:00


In [9]:
df.order_purchase_timestamp = pd.to_datetime(df['order_purchase_timestamp'], 
                                            format= '%Y-%m-%d %H:%M:%S', 
                                            errors="coerce")

df.order_delivered_customer_date = pd.to_datetime(df['order_delivered_customer_date'], 
                                                  format= '%Y-%m-%d %H:%M:%S', 
                                                  errors="coerce")

df.order_estimated_delivery_date = pd.to_datetime(df['order_estimated_delivery_date'], 
                                                  format= '%Y-%m-%d %H:%M:%S', 
                                                  errors="coerce")


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98344 entries, 0 to 98343
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   review_id                      98344 non-null  object        
 1   order_id                       98344 non-null  object        
 2   review_score                   98344 non-null  int64         
 3   review_comment_title           98344 non-null  object        
 4   review_comment_message         98344 non-null  object        
 5   review_creation_date           98344 non-null  object        
 6   review_answer_timestamp        98344 non-null  object        
 7   customer_id                    98344 non-null  object        
 8   order_status                   98344 non-null  object        
 9   order_purchase_timestamp       98344 non-null  datetime64[ns]
 10  order_approved_at              98344 non-null  object        
 11  order_delivered

In [12]:
df.loc[df.order_delivered_customer_date.isna(),:]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
41,6d06808638ec0701bccd70bc8d462c28,97d2f8fe76f2f253b8291e17b5383884,1,,O produto não chegou no prazo estipulado e cau...,2016-12-09 00:00:00,2016-12-19 14:10:43,5458c93e0db88f9b885bc7dbc3bfa2a3,shipped,2016-10-05 11:54:15,2016-10-06 15:51:26,2016-11-14 09:28:51,NaT,2016-12-07
62,65dfeb60c40e3cbb0a1838285d86f885,a2714ecbf6eeb3bb9cd7dba6dc1c5e82,1,,Pedi reembolso e sem resposta até momento,2018-03-08 00:00:00,2018-03-08 12:27:34,5fe0562353e8f0b6c0c6b55cd0f58fe0,canceled,2018-02-14 18:01:38,2018-02-14 18:20:40,2018-02-15 17:27:27,NaT,2018-03-06
77,352609f0f2d34ce5d3b8de817ee948d5,96b947c986b42f213c9e6b8ccccedb89,3,Não recebi o produto.,,2018-08-25 00:00:00,2018-08-25 20:53:41,0be97ebc9313a86e02d3b7cdd7e5f1a6,canceled,2018-07-30 13:00:56,2018-07-31 03:55:23,,NaT,2018-08-22
142,185cb85ede91009d7a8e685687c037ee,2e5dc86c8c4aa663549caf5e31de840d,1,,fiz minha compra faz 30 dias e não recebi aind...,2017-04-06 00:00:00,2017-04-09 18:25:12,6225eed02b7d1a110b6e5b5dd4c8bd31,processing,2017-02-03 00:04:49,2017-04-04 10:56:48,,NaT,2017-03-31
143,0c9b6ca9b3beaf0fd5d621f63844ea01,cadd4c5765abf658117f2cc7e2a639c2,5,prazo entrega,"Espero receber esta semana, o que não seria tã...",2018-08-26 00:00:00,2018-08-29 02:48:21,939c6ecf27d8c32e7956b4a2a468326d,canceled,2018-08-10 21:20:04,2018-08-10 21:30:18,,NaT,2018-08-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98167,e1bc9e2f33c3fa1cb40845c285f0c02b,dd359d3c294458c6d642b2eea9212bf5,2,,"Já comprei várias vezes no site ""lannister"";ma...",2016-11-27 00:00:00,2016-11-29 14:19:59,5c58d1ea5a893380ecdd96dd6dfd5ec5,invoiced,2016-10-04 13:02:10,2016-10-05 03:08:27,,NaT,2016-11-24
98168,71fd2bd9edec8a1cd368a5fa173979d6,4a5cf649e3eb500fc079aace5f03583e,1,,Não cumpriu o prazo.. Não teve capacidade para...,2018-03-30 00:00:00,2018-03-30 21:52:35,128433c9283a98b6eddefd309edc7e00,unavailable,2018-03-05 14:37:40,2018-03-05 14:55:39,,NaT,2018-03-28
98278,2fc8295a24978092328d3a329d22940e,e809c167a9cfd31aee1293abe8995065,1,,Eu recebi o seguinte email e preciso saber com...,2018-02-09 00:00:00,2018-02-13 13:03:24,9472f9fca02c86c72be6429983d0a76a,processing,2018-01-04 18:23:45,2018-01-06 02:08:26,,NaT,2018-02-05
98293,314d2e4c14be2030c24c3f29ff3740a7,e3c4b2d7022588a02583a79b3f405632,5,,Satisfeita,2017-04-22 00:00:00,2017-04-22 10:38:17,ffb81db92e7ac00ecfac978f673be8a6,shipped,2017-03-17 21:14:36,2017-03-17 21:14:36,2017-03-20 09:49:10,NaT,2017-04-20


In [15]:
df_orders.loc[df_orders.order_id == 'cadd4c5765abf658117f2cc7e2a639c2',:]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
58108,cadd4c5765abf658117f2cc7e2a639c2,939c6ecf27d8c32e7956b4a2a468326d,canceled,2018-08-10 21:20:04,2018-08-10 21:30:18,,,2018-08-24 00:00:00


Pour le moment on garde les lignes sans valeur de livraison NA pour pouvoir les identifier dans la suite de l'analyse

## 2.4 Consistence des données

In [6]:
df.order_status.value_counts()

order_status
delivered      95544
shipped         1027
unavailable      590
canceled         576
invoiced         308
processing       294
created            3
approved           2
Name: count, dtype: int64

# 3 Jointure et nettoyage de la table Order_item

In [10]:
connection = sqlite3.connect("olist.db")

df = pd.read_sql_query("SELECT * FROM CleanDataset",connection)

df_order_item = pd.read_sql_query("SELECT * FROM OrderItem",connection)

connection.close()

In [24]:
df_montant_global = df_order_item[["order_id","price","freight_value"]].groupby("order_id").sum()

In [25]:
df_montant_global.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98666 entries, 00010242fe8c5a6d1ba2dd792cb16214 to fffe41c64501cc87c801fd61db3f6244
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          98666 non-null  float64
 1   freight_value  98666 non-null  float64
dtypes: float64(2)
memory usage: 2.3+ MB


In [16]:
df = df.merge(df_montant_global, how='left', on ='order_id')

## 2.1 Gestion des valeurs manquantes

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98344 entries, 0 to 98343
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   review_id                      98344 non-null  object 
 1   order_id                       98344 non-null  object 
 2   review_score                   98344 non-null  int64  
 3   review_comment_title           98344 non-null  object 
 4   review_comment_message         98344 non-null  object 
 5   review_creation_date           98344 non-null  object 
 6   review_answer_timestamp        98344 non-null  object 
 7   customer_id                    98344 non-null  object 
 8   order_status                   98344 non-null  object 
 9   order_purchase_timestamp       98344 non-null  object 
 10  order_approved_at              98344 non-null  object 
 11  order_delivered_carrier_date   98344 non-null  object 
 12  order_delivered_customer_date  95542 non-null 

In [20]:
df.loc[df.price.isna(),:].order_status.value_counts()

order_status
unavailable    584
canceled       135
created          3
invoiced         2
shipped          1
Name: count, dtype: int64

## 2.2 Gestion des doublons et des types

OK

## 2.3 Gestion de la consistance

In [23]:
df[["price","freight_value"]].describe()

,price,freight_value
count,97619.000000,97619.000000
mean,137.568367,22.805388
std,209.456318,21.634085
min,0.850000,0.000000
25%,45.900000,13.840000
50%,86.900000,17.170000
75%,149.900000,24.010000
max,13440.000000,1794.960000


# 4 Jointure et nettoyage de la table Products

Etapes pour construire "qty moyenne de photos" pour ma commande (chaque commande est rattachée à un commentaire):
- importer les tables
- Join entre order_item et product pour obtenir une table avec les information de product et la cardinalité de order_item c'est à dire plusieurs lignes par order_id (commande)
- On réduit la cardinalité en faisant un group by order_id et comme ça on obtient une ligne par order_id, la cardinalité qu'on souhaitait. POur synthétiser la colonne QTY on prend la moyenne des différents items pour un order_id
- On fait enfin un join entre clean et la table crée après le groupby


In [ ]:
clean
order_id, score,   

df_order_item
order_id, product_id

product
product_id, QTY

In [21]:
connection = sqlite3.connect("olist.db")

df = pd.read_sql_query("SELECT * FROM CleanDataset",connection)

df_order_item = pd.read_sql_query("SELECT * FROM OrderItem",connection)

df_products = pd.read_sql_query("SELECT * FROM Products",connection)

connection.close()

In [4]:
df.shape

(98344, 16)

In [5]:
df_order_item.shape

(112650, 7)

In [6]:
df_products.shape

(32951, 9)

In [22]:
df_item_product = df_order_item[['order_id','product_id']].merge(df_products, how='left', on ='product_id')

In [9]:
df_item_product.shape

(112650, 10)

In [10]:
df_item_product.columns

Index(['order_id', 'product_id', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm'],
      dtype='object')

In [23]:
df_item_product["product_photos_qty"] = df_item_product["product_photos_qty"].replace("","0").astype("int")

In [32]:
df_item_product["product_description_lenght"] = df_item_product["product_description_lenght"].replace("","0").astype("int")


In [20]:
df.shape

(112108, 25)

In [16]:
df_item_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   order_id                    112650 non-null  object
 1   product_id                  112650 non-null  object
 2   product_category_name       112650 non-null  object
 3   product_name_lenght         112650 non-null  object
 4   product_description_lenght  112650 non-null  object
 5   product_photos_qty          112650 non-null  int64 
 6   product_weight_g            112650 non-null  object
 7   product_length_cm           112650 non-null  object
 8   product_height_cm           112650 non-null  object
 9   product_width_cm            112650 non-null  object
dtypes: int64(1), object(9)
memory usage: 8.6+ MB


In [33]:
df_item_product_group_by = df_item_product[['order_id','product_photos_qty','product_description_lenght']].groupby('order_id').mean()

In [34]:
df_item_product_group_by = df_item_product_group_by.reset_index()

In [35]:
df_item_product_group_by.head()

,order_id,product_photos_qty,product_description_lenght
0,00010242fe8c5a6d1ba2dd792cb16214,4.0,598.0
1,00018f77f2f0320c557190d7a144bdd3,2.0,239.0
2,000229ec398224ef6ca0657da4fc703e,2.0,695.0
3,00024acbcdf0a6daa1e931b038114c75,1.0,480.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,409.0


In [36]:
df = df.merge(df_item_product_group_by,how='left')

In [30]:
df.shape

(98344, 17)

In [37]:
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,price,freight_value,product_photos_qty,product_description_lenght
0,fe3db7c069d694bab50cc43463f91608,2ca73e2ff9e3a186ad1e1ffb9b1d9c10,5,,A compra foi realizada facilmente.\r\nA entreg...,2018-03-23 00:00:00,2018-04-01 00:27:51,e76f777f29e83b393d786accf49e7955,delivered,2018-03-18 21:01:32,2018-03-18 21:15:36,2018-03-19 19:52:47,2018-03-23 00:32:34,2018-04-11 00:00:00,139.9,39.97,3.0,1184.0
1,e07eee7ad7666276bd8b27fbd83fd1ff,68b3cbc68e4cd1fe19bec35f1d3bb7ee,5,,relógio muito bonito e barato.,2018-03-29 00:00:00,2018-04-02 14:19:31,1a814d4c8923ef3a0e978777f00b717c,delivered,2018-03-10 00:42:20,2018-03-10 01:21:04,2018-03-12 21:05:34,2018-03-28 18:13:06,2018-03-29 00:00:00,65.0,15.33,7.0,221.0
2,2c5e27fc178bde7ac173c9c62c31b070,0ce9a24111d850192a933fcaab6fbad3,1,,Não gostei ! Comprei gato por lebre,2017-12-13 00:00:00,2017-12-16 07:14:07,5bb8de60ca2ca8b01a5ce471802fe10b,delivered,2017-11-24 01:40:48,2017-11-24 01:49:34,2017-12-06 15:19:09,2017-12-13 00:28:44,2017-12-19 00:00:00,159.9,28.50,1.0,729.0
3,540e7bbb2d06cfb7f85f3a88ba7ac97f,cf73e2cb1f4a9480ed70c154da3d954a,5,,,2018-01-18 00:00:00,2018-01-18 19:12:30,ef1ed1347404166e73e8f3977ffcbca0,delivered,2018-01-11 11:23:09,2018-01-12 02:38:33,2018-01-13 01:07:00,2018-01-17 14:36:38,2018-02-01 00:00:00,50.0,11.85,1.0,558.0
4,6a18a4001ae7fd911b3b9fb13686d105,8ba3f3a9292fbc22a8b7dff752bbcd37,5,,,2017-07-19 00:00:00,2017-07-20 10:38:48,0315a87fba4dab4865f2071777b98e87,delivered,2017-07-07 10:07:16,2017-07-07 10:23:34,2017-07-08 11:46:44,2017-07-18 23:16:44,2017-07-31 00:00:00,59.9,16.18,3.0,1266.0


In [ ]:
df_cat_prod

In [62]:
pd.get_dummies(df_cat_prod["product_category_name"]).shape

(112650, 74)

In [66]:
df_cat_prod[category_list] = pd.get_dummies(df_cat_prod["product_category_name"])

In [75]:
df_cat_prod = df_cat_prod.drop(['product_id','product_category_name','product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm'],axis=1)

In [76]:
df_cat_prod.head()

,order_id,cool_stuff,pet_shop,moveis_decoracao,perfumaria,ferramentas_jardim,utilidades_domesticas,telefonia,beleza_saude,livros_tecnicos,...,cds_dvds_musicais,artes_e_artesanato,moveis_colchao_e_estofado,tablets_impressao_imagem,construcao_ferramentas_ferramentas,fashion_esporte,la_cuisine,pc_gamer,seguros_e_servicos,fashion_roupa_infanto_juvenil
0,00010242fe8c5a6d1ba2dd792cb16214,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,00018f77f2f0320c557190d7a144bdd3,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,000229ec398224ef6ca0657da4fc703e,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,00024acbcdf0a6daa1e931b038114c75,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,00042b26cf59d7ce69dfabb4e55b4fd9,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [83]:
df_cat_prod_by_order = df_cat_prod.groupby("order_id").max()

In [84]:
df_cat_prod_by_order

,cool_stuff,pet_shop,moveis_decoracao,perfumaria,ferramentas_jardim,utilidades_domesticas,telefonia,beleza_saude,livros_tecnicos,fashion_bolsas_e_acessorios,...,cds_dvds_musicais,artes_e_artesanato,moveis_colchao_e_estofado,tablets_impressao_imagem,construcao_ferramentas_ferramentas,fashion_esporte,la_cuisine,pc_gamer,seguros_e_servicos,fashion_roupa_infanto_juvenil
order_id,,,,,,,,,,,,,,,,,,,,,
00010242fe8c5a6d1ba2dd792cb16214,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
00018f77f2f0320c557190d7a144bdd3,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
000229ec398224ef6ca0657da4fc703e,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
00024acbcdf0a6daa1e931b038114c75,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
00042b26cf59d7ce69dfabb4e55b4fd9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffc94f6ce00a00581880bf54a75a037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
fffcd46ef2263f404302a634eb57f7eb,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
fffce4705a9662cd70adb13d4a31832d,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [86]:
# Calculate the row sum
df_cat_prod_by_order.sum(axis=1).value_counts()



1    97880
2      768
3       18
Name: count, dtype: int64

In [ ]:
# Add a new column with the row sum to the DataFrame
df['RowSum'] = row_sum

In [47]:
df_order_item.loc[df_order_item['order_id']=="428a2f660dc84138d969ccd69a0ab6d5",:]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
29257,428a2f660dc84138d969ccd69a0ab6d5,1,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29258,428a2f660dc84138d969ccd69a0ab6d5,2,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29259,428a2f660dc84138d969ccd69a0ab6d5,3,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29260,428a2f660dc84138d969ccd69a0ab6d5,4,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29261,428a2f660dc84138d969ccd69a0ab6d5,5,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29262,428a2f660dc84138d969ccd69a0ab6d5,6,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29263,428a2f660dc84138d969ccd69a0ab6d5,7,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29264,428a2f660dc84138d969ccd69a0ab6d5,8,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29265,428a2f660dc84138d969ccd69a0ab6d5,9,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22
29266,428a2f660dc84138d969ccd69a0ab6d5,10,89b190a046022486c635022524a974a8,f326006815956455b2859abd58fe7e39,2017-11-30 10:30:51,65.49,16.22


In [28]:
df = df.merge(df_order_item[['order_id','product_id']], how='left', on ='order_id')

In [29]:
df = df.merge(df_products, how='left', on ='product_id')

## 4.1 Gestion des valeurs manquantes

In [32]:
df.iloc[:,15:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112108 entries, 0 to 112107
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   freight_value               111383 non-null  float64
 1   product_id                  111383 non-null  object 
 2   product_category_name       111383 non-null  object 
 3   product_name_lenght         111383 non-null  object 
 4   product_description_lenght  111383 non-null  object 
 5   product_photos_qty          111383 non-null  object 
 6   product_weight_g            111383 non-null  object 
 7   product_length_cm           111383 non-null  object 
 8   product_height_cm           111383 non-null  object 
 9   product_width_cm            111383 non-null  object 
dtypes: float64(1), object(9)
memory usage: 8.6+ MB


## 4.2 Gestion des doublons et des types

## 4.3 Gestion des types

In [38]:
import numpy as np

In [41]:
df['product_photos_qty'] = df['product_photos_qty'].replace([np.nan,''], '0').astype(int)
df['product_description_lenght'] = df['product_description_lenght'].replace([np.nan,''], '0').astype(int)

OK

## 4.4 Gestion de la consistance

In [33]:
df.product_category_name.value_counts()

product_category_name
cama_mesa_banho                  10944
beleza_saude                      9558
esporte_lazer                     8545
moveis_decoracao                  8211
informatica_acessorios            7749
                                 ...  
cds_dvds_musicais                   14
la_cuisine                          13
pc_gamer                             9
fashion_roupa_infanto_juvenil        8
seguros_e_servicos                   2
Name: count, Length: 74, dtype: int64

In [42]:
df[["product_photos_qty","product_description_lenght"]].describe()

,product_photos_qty,product_description_lenght
count,112108.000000,112108.000000
mean,2.165162,771.610777
std,1.733597,654.737311
min,0.000000,0.000000
25%,1.000000,340.000000
50%,1.000000,591.000000
75%,3.000000,975.000000
max,20.000000,3992.000000


In [38]:
connection = sqlite3.connect("olist.db")

df_customers = pd.read_sql_query("SELECT * FROM Customers",connection)

connection.close()

In [42]:
v_count = df_customers['customer_unique_id'].value_counts()

In [45]:
v_count[v_count.values>1].shape

(2997,)

In [46]:
v_count[v_count.values>1].sum()

6342